In [3]:
# =========================================================
# Brightness temperature from chromosphere, corona and outer corona
# =========================================================

import numpy as np
from scipy.integrate import simps


CFF = 9.786e-3   # free–free absorption coefficient prefactor (cgs)


# ---------------------------------------------------------
# Coulomb logarithm
# Te in K, nu in Hz
# ---------------------------------------------------------
def coulomb_logarithm(Te, nu):
    return 24.5 + np.log(Te) - np.log(nu)


# ---------------------------------------------------------
# Free–free absorption coefficient alpha_ff (cm^-1)
# ne, ni in cm^-3, Te in K, nu in Hz
# ---------------------------------------------------------
def alpha_ff(ne, Te, nu, Zi=1.0):
    lnL = coulomb_logarithm(Te, nu)
    ni = ne  # quasi-neutral plasma
    return (CFF * ne * ni * Zi**2 * lnL) / (nu**2 * Te**1.5)


# ---------------------------------------------------------
# Optical depth tau(z) = ∫_z^0 alpha(z') dz'
# Observer at z = 0
# ---------------------------------------------------------
def optical_depth_global(z, alpha):
    tau = np.zeros_like(z)

    # integrate from each z-point to observer
    for i in range(len(z)):
        tau[i] = simps(alpha[i:], z[i:])

    return tau


# ---------------------------------------------------------
# Brightness temperature from a z-segment
# ---------------------------------------------------------
def brightness_temperature_segment(z, ne, Te, nu, zmin, zmax, tau_global, Zi=1.0):
    mask = (z >= zmin) & (z <= zmax)

    z_seg   = z[mask]
    ne_seg  = ne[mask]
    Te_seg  = Te[mask]
    tau_seg = tau_global[mask]

    alpha_seg = alpha_ff(ne_seg, Te_seg, nu, Zi)

    integrand = Te_seg * np.exp(-tau_seg) * alpha_seg

    TB = simps(integrand, z_seg)
    return TB


# =========================================================
# MAIN SCRIPT
# =========================================================

# ---------------------------------------------------------
# LOS grid (cm), observer at z = 0
# ---------------------------------------------------------
z = np.linspace(-5e10, 0.0, 4000)

# ---------------------------------------------------------
# Plasma profiles (simple model)
# ---------------------------------------------------------
ne = 1e8 * np.exp(z / 2e9)          # cm^-3
Te = 1.5e6 * np.ones_like(z)        # K

# Observing frequency
nu = 100e6  # 100 MHz

# ---------------------------------------------------------
# Compute global optical depth
# ---------------------------------------------------------
alpha = alpha_ff(ne, Te, nu)
tau_global = optical_depth_global(z, alpha)

# ---------------------------------------------------------
# Atmospheric layer boundaries (cm)
# ---------------------------------------------------------
z_chromo_min = -2e8
z_chromo_max = 0.0

z_corona_min = -1e10
z_corona_max = -2e8

z_outer_min  = -5e10
z_outer_max  = -1e10

# ---------------------------------------------------------
# Brightness temperatures
# ---------------------------------------------------------
TB_chromo = brightness_temperature_segment(
    z, ne, Te, nu,
    z_chromo_min, z_chromo_max,
    tau_global
)

TB_corona = brightness_temperature_segment(
    z, ne, Te, nu,
    z_corona_min, z_corona_max,
    tau_global
)

TB_outer = brightness_temperature_segment(
    z, ne, Te, nu,
    z_outer_min, z_outer_max,
    tau_global
)

TB_total = TB_chromo + TB_corona + TB_outer

# ---------------------------------------------------------
# Output
# ---------------------------------------------------------
print("Frequency = %.1f MHz" % (nu / 1e6))
print("---------------------------------------")
print("TB (Chromosphere) = %.3e K" % TB_chromo)
print("TB (Corona)       = %.3e K" % TB_corona)
print("TB (Outer Corona) = %.3e K" % TB_outer)
print("---------------------------------------")
print("TB (Total)        = %.3e K" % TB_total)


Frequency = 100.0 MHz
---------------------------------------
TB (Chromosphere) = 2.748e+04 K
TB (Corona)       = 1.246e+05 K
TB (Outer Corona) = 6.593e+00 K
---------------------------------------
TB (Total)        = 1.521e+05 K
